In [1]:
from src.utils.spark_utils import SparkUtils 

# create a spark session 
spark = SparkUtils(app_name="nyc-taxi-data-ingestion").spark

# read the data from the parquet file 
df = spark.read.parquet("gold/2025/january/trip_01.parquet")

df.show()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/09 02:31:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/09 02:31:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/09 02:32:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------+------------+--------+--------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+------------------+-----------+-------------------+-------------------+------------+-----------+------------------+----------+------------+--------+------------+-------------------+------------------+------------------+--------------------+-------------+------------+-------------------+------------------+------------+----------------------+-----------------------+----------------+------------------+--------------------+--------------------+---------+---------------+------------------+------------------+
|route_id|DOLocationID|PULocationID|VendorID|tpep_pickup_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcha

In [4]:
## count the number of rows in dataframe 
df.count()





3250902

26/02/09 04:52:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 221714 ms exceeds timeout 120000 ms
26/02/09 04:52:44 WARN SparkContext: Killing executors is not supported by current scheduler.
26/02/09 04:52:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [3]:
df.show(5)

+--------+------------+------------+--------+--------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+------------------+-----------+-------------------+-------------------+------------+-----------+------------------+----------+------------+--------+------------+--------+--------+------------------+--------------------+-------------+------------+------------------+------------------+------------+----------------------+-----------------------+----------------+------------------+-------------------+----------+---------+---------------+------------------+------------------+
|route_id|DOLocationID|PULocationID|VendorID|tpep_pickup_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|Airport_fee|cbd_congestion_fe